In [1]:
!pip install tensorflow keras opencv-python pillow numpy


In [6]:
# 📦 Install dependencies
!pip install tensorflow keras opencv-python pillow numpy scikit-learn

# 📂 Import libraries
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# 📁 Paths
images_path = r"C:\Users\sagni\Downloads\HandWriting to Digital Notes\archive (1)\self_lines"
labels_path = r"C:\Users\sagni\Downloads\HandWriting to Digital Notes\archive (1)\lines.txt"
model_save_path = r"C:\Users\sagni\Downloads\HandWriting to Digital Notes\ocr_model.h5"

# 📥 Load labels
with open(labels_path, "r", encoding="utf-8") as f:
    raw_labels = [line.strip() for line in f.readlines()]

# 🖼️ Load and preprocess images
image_files = sorted(os.listdir(images_path))
images, labels = [], []
min_samples = min(len(image_files), len(raw_labels))
for i in range(min_samples):
    img_path = os.path.join(images_path, image_files[i])
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"⚠️ Skipped unreadable image: {image_files[i]}")
        continue
    img = cv2.resize(img, (128, 32))
    img = img / 255.0
    images.append(img)
    labels.append(raw_labels[i])
images = np.array(images).reshape(-1, 32, 128, 1)
print(f"✅ Loaded {len(images)} images aligned with {len(labels)} labels.")

# 🔤 Encode labels
all_chars = sorted(set("".join(labels)))
char_to_num = {char: idx + 1 for idx, char in enumerate(all_chars)}
num_to_char = {idx + 1: char for idx, char in enumerate(all_chars)}
vocab_size = len(char_to_num) + 2  # +1 blank token +1 padding

max_label_len = max(len(label) for label in labels)
encoded_labels = np.zeros((len(labels), max_label_len))
label_lengths = np.zeros((len(labels), 1), dtype=np.int32)
for i, label in enumerate(labels):
    label_encoded = [char_to_num[char] for char in label]
    encoded_labels[i, :len(label_encoded)] = label_encoded
    label_lengths[i] = len(label_encoded)

# 🪜 Split dataset
X_train, X_val, y_train, y_val, len_train, len_val = train_test_split(
    images, encoded_labels, label_lengths, test_size=0.1, random_state=42
)

# 🏗️ Build CRNN Base Model
inputs = keras.Input(shape=(32, 128, 1), name="input_image")
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(inputs)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
new_shape = (x.shape[1] * x.shape[2], x.shape[3])
x = layers.Reshape(target_shape=new_shape)(x)
x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
base_model = keras.Model(inputs, outputs, name="crnn_base")
print("✅ Base CRNN model built.")

# 📦 CTC Loss Layer
labels_input = keras.Input(shape=(max_label_len,), name="labels")
input_lengths = keras.Input(shape=(1,), name="input_lengths")
label_lengths_input = keras.Input(shape=(1,), name="label_lengths")

def ctc_lambda_func(args):
    y_pred, labels, input_len, label_len = args
    return keras.backend.ctc_batch_cost(labels, y_pred, input_len, label_len)

loss_output = layers.Lambda(ctc_lambda_func, name="ctc_loss")(
    [outputs, labels_input, input_lengths, label_lengths_input])

# 🎯 Training model (with dummy loss)
training_model = keras.Model(
    inputs=[inputs, labels_input, input_lengths, label_lengths_input],
    outputs=loss_output)
training_model.compile(optimizer="adam", loss=lambda y_true, y_pred: y_pred)
print("✅ Training model compiled with dummy loss.")

# 📊 Data for CTC
train_input_lengths = np.ones((X_train.shape[0], 1)) * (outputs.shape[1])
val_input_lengths = np.ones((X_val.shape[0], 1)) * (outputs.shape[1])

# 🚀 Train
history = training_model.fit(
    x=[X_train, y_train, train_input_lengths, len_train],
    y=np.zeros(len(X_train)),
    validation_data=(
        [X_val, y_val, val_input_lengths, len_val],
        np.zeros(len(X_val))
    ),
    epochs=25,
    batch_size=16
)

# 💾 Save prediction model (base model)
base_model.save(model_save_path)
print(f"🎉 Saved prediction model at: {model_save_path}")

# ✅ Load for testing
loaded_model = keras.models.load_model(model_save_path)
print("🔄 Model loaded for inference.")


✅ Loaded 336 images aligned with 336 labels.
✅ Base CRNN model built.
✅ Training model compiled with dummy loss.
Epoch 1/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 165ms/step - loss: 560.9841 - val_loss: 246.5246
Epoch 2/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 123ms/step - loss: 238.3772 - val_loss: 229.1406
Epoch 3/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 124ms/step - loss: 230.0960 - val_loss: 226.0607
Epoch 4/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - loss: 227.9818 - val_loss: 220.7463
Epoch 5/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 124ms/step - loss: 220.0516 - val_loss: 209.2045
Epoch 6/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 122ms/step - loss: 206.0189 - val_loss: 192.3601
Epoch 7/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - loss: 189.0139 - val_loss: 175.6801
Epoch 8/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - loss: 173.6223 - val_loss: 166.4435
Epoch 9/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - loss: 164.7584 - val_loss: 158.8198
Epoch 10/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 123ms/step - loss: 158.1279 - val_l

🎉 Saved prediction model at: C:\Users\sagni\Downloads\HandWriting to Digital Notes\ocr_model.h5
🔄 Model loaded for inference.
